In [12]:
import numpy as np

In [13]:
def sigmoid(x):
    return 1/ (1+np.exp(-x))

In [14]:

x = np.random.randn(10, 2)
W1= np.random.randn(2, 4)
b1 = np.random.randn(4)
W2= np.random.randn(4, 3)
b2 = np.random.randn(3)

In [15]:
h = np.dot(x, W1) + b1

In [16]:
a = sigmoid(h)

In [17]:
s = np.dot(a, W2) +b2

In [19]:
s

array([[ 1.08285728,  2.02263511, -0.06264663],
       [ 1.14509711,  2.11309067,  0.10351852],
       [ 1.45600798,  2.78811041,  0.41065505],
       [ 1.08559167,  1.99529233, -0.07268861],
       [ 1.16511869,  2.18635774,  0.13207122],
       [ 1.22205138,  2.25294846,  0.29650455],
       [ 0.93620385,  1.79109163, -0.45339328],
       [ 1.01000719,  1.90565137, -0.25683554],
       [ 0.87801921,  1.80790965, -0.74783178],
       [ 1.44840624,  2.72512972,  0.53021923]])